# 0. Load imports 

In [1]:
## imports
import pandas as pd
import numpy as np
import re

## print multiple things from same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## load data on 2020 crimes in DC
df = dc_crim_2020 = pd.read_csv("https://opendata.arcgis.com/datasets/f516e0dd7b614b088ad781b0c4002331_2.csv")

# 1. Questions: list comprehension

- In class example, why did we need the "courses" at the beginning of the list iteration
- How did the join syntax work in the example where we paste together offenses from same ward

In [2]:
## toy example

### pool of courses
all_courses = ["QSS20", "QSS17", "GOV10", "GOV4", "CSC1"]

## 1.1 Application 1: filtering to a smaller list

When we might use: have a lot of columns in a dataframe; want to filter to a smaller set using some pattern

In [3]:
### pull out ones that contain GOV in the string
gov_c = [course for course in all_courses
        if "GOV" in course]
gov_c # result

['GOV10', 'GOV4']

In [4]:
### showing that the "course" is just a placeholder/
### arbitrary interator
gov_c_alt = [x for x in all_courses if "GOV" in x]

gov_c == gov_c_alt

True

## 1.2 Application two: keep all objects in the list but do some transformation

In [5]:
all_courses

## strip the numbers from the course names
courses_prefix = [x[:3] for x in all_courses]
courses_prefix # could then find unique elements


['QSS20', 'QSS17', 'GOV10', 'GOV4', 'CSC1']

['QSS', 'QSS', 'GOV', 'GOV', 'CSC']

In [6]:
# Join all together example
" #:)# ".join(courses_prefix)

'QSS #:)# QSS #:)# GOV #:)# GOV #:)# CSC'

#### Your turn: Using original list, add "dartmouth_" prefix to the course name

In [11]:
["dartmouth_" + c for c in all_courses]

['dartmouth_QSS20',
 'dartmouth_QSS17',
 'dartmouth_GOV10',
 'dartmouth_GOV4',
 'dartmouth_CSC1']

## 1.3 Subsetting columns

Use list comprehension to filter to columns with id in the string. Then, create a new dataframe called df1 that contains only column heads with "id"

In [12]:
id_cols = [col for col in df.columns if "ID" in col]
id_cols

## Then, filter the data
df[id_cols]

['BID', 'OBJECTID', 'OCTO_RECORD_ID']

,BID,OBJECTID,OCTO_RECORD_ID
0,NaN,595556840,NaN
1,DOWNTOWN,595560645,NaN
2,NaN,595560647,NaN
3,GOLDEN TRIANGLE,595560654,NaN
4,NaN,595560828,NaN
...,...,...,...
27926,NaN,596085252,NaN
27927,NaN,596085253,NaN
27928,NaN,596085258,NaN
27929,DOWNTOWN,596085259,NaN


## 1.4 Comprehension for numbers

Here we compare two ways of creating a list of even numbers.

In [13]:
num_list = np.arange(10000)
num_list

array([   0,    1,    2, ..., 9997, 9998, 9999])

In [16]:
%%time
even_nums = [i for i in num_list if (i % 2) == 0]

CPU times: user 1.61 ms, sys: 88 μs, total: 1.7 ms
Wall time: 1.71 ms


In [17]:
%%time
num_list[~(num_list % 2).astype(bool)]

CPU times: user 629 μs, sys: 220 μs, total: 849 μs
Wall time: 624 μs


array([   0,    2,    4, ..., 9994, 9996, 9998])

#### Your turn: Extract all numbers in num_list that end in 7

In [21]:
%%time
seven = [x for x in num_list if x%10 == 7]

CPU times: user 1.54 ms, sys: 1 μs, total: 1.54 ms
Wall time: 1.55 ms


In [23]:
%%time
seven = num_list[(num_list % 10) == 7]

CPU times: user 747 μs, sys: 842 μs, total: 1.59 ms
Wall time: 944 μs


#### Your turn: Divide each number  in num_list by 2

In [28]:
%%time
half = num_list/2

CPU times: user 500 μs, sys: 177 μs, total: 677 μs
Wall time: 605 μs


1.0

In [31]:
%%time
half = [x/2 for x in num_list]

CPU times: user 1.32 ms, sys: 14 μs, total: 1.34 ms
Wall time: 1.35 ms


0.5

# 2. Questions: lambda functions

Two questions:

- General syntax (see here for a reference: https://www.w3schools.com/python/python_lambda.asp 
- How they work in the context of aggregations

How is a lambda function different from a "normal" user-defined function (that has the syntax def func_name(arg): etc?

- Operates similarly to normal user-defined functions in that it can take any # of arguments
- Operates differently in that it's an "anonymous" function or a function that we don't explicitly name/save in memory

In [32]:
def f1(x,y):
    return x+y

f2 = lambda x, y: x+y

f1(2,1)
f2(2,1)

3

3

## 2.1 General syntax for lambda functions

In [33]:
### two pools of courses
socsci = ["QSS20", "QSS17", "GOV10"]
natsci = ["BIO2", "PHYS3"]


## generalize some of the steps
## above into a two-arg function
## that takes the course prefix
## and a list of all courses
def filter_courses(prefix,all_courses):
    rel_courses = [c for c in all_courses if prefix in c]
    return(rel_courses)

### a few applications 
filter_courses(prefix = "QSS", all_courses = socsci)
filter_courses(prefix = "QSS", all_courses = natsci)
filter_courses(prefix = "BIO", all_courses = natsci)

['QSS20', 'QSS17']

[]

['BIO2']

In [34]:
## what's the lambda function version of this
filter_courses_v2 = lambda prefix, all_courses: [c for c in all_courses if prefix in c]
filter_courses_v2(prefix = "BIO", all_courses = natsci)


['BIO2']

## 2.2 using alongside agg

In [35]:
## use lambda to find modal block in a ward- multiple ways

### way 1: subsetting agg syntex
df.groupby("WARD")["BLOCK"].agg(lambda x: x.mode())

### way 2: dictionary agg syntax
df.groupby("WARD").agg({
    "BLOCK": lambda x: x.mode()
})


WARD
1                  3100 - 3299 BLOCK OF 14TH STREET NW
2    [1100 - 1199 BLOCK OF NEW YORK AVENUE NW, 1300...
3             5300 - 5399 BLOCK OF WISCONSIN AVENUE NW
4                100  - 199 BLOCK OF CARROLL STREET NW
5            900 - 999 BLOCK OF RHODE ISLAND AVENUE NE
6                       600 - 699 BLOCK OF H STREET NE
7                934 - 1099 BLOCK OF EASTERN AVENUE NE
8               2300 - 2399 BLOCK OF GOOD HOPE ROAD SE
Name: BLOCK, dtype: object

,BLOCK
WARD,
1,3100 - 3299 BLOCK OF 14TH STREET NW
2,"[1100 - 1199 BLOCK OF NEW YORK AVENUE NW, 1300..."
3,5300 - 5399 BLOCK OF WISCONSIN AVENUE NW
4,100 - 199 BLOCK OF CARROLL STREET NW
5,900 - 999 BLOCK OF RHODE ISLAND AVENUE NE
6,600 - 699 BLOCK OF H STREET NE
7,934 - 1099 BLOCK OF EASTERN AVENUE NE
8,2300 - 2399 BLOCK OF GOOD HOPE ROAD SE


#### Your turn: Group by WARD and get the mean and standard deviation (std) of X and Y

In [38]:
df.groupby("WARD").agg({
    "X": ["mean", "std"],
    "Y": ["mean", "std"],
})

X                    Y          
           mean       std       mean       std
WARD                                          
1    -77.031256  0.007200  38.924716  0.006417
2    -77.038959  0.014106  38.905941  0.005360
3    -77.075320  0.013408  38.941557  0.013887
4    -77.024905  0.011813  38.957674  0.013923
5    -76.989451  0.014957  38.920249  0.012766
6    -77.001175  0.011302  38.893112  0.010021
7    -76.947402  0.018753  38.888669  0.012504
8    -76.988986  0.012791  38.850526  0.015775

In [48]:
df.groupby("WARD")[["X","Y"]].agg(["mean", "std"])

X                    Y          
           mean       std       mean       std
WARD                                          
1    -77.031256  0.007200  38.924716  0.006417
2    -77.038959  0.014106  38.905941  0.005360
3    -77.075320  0.013408  38.941557  0.013887
4    -77.024905  0.011813  38.957674  0.013923
5    -76.989451  0.014957  38.920249  0.012766
6    -77.001175  0.011302  38.893112  0.010021
7    -76.947402  0.018753  38.888669  0.012504
8    -76.988986  0.012791  38.850526  0.015775